# Imports

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import os
import sys

import mujoco
import mediapy as media
from gym import Env
from gym.spaces import Discrete, Box

import stable_baselines3
from stable_baselines3.common.env_checker import check_env
import torch as th
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.logger import configure
from stable_baselines3.common.logger import Logger, CSVOutputFormat, HumanOutputFormat

# Cargar los datos de captura de movimiento y simulación

In [2]:
datos = pd.read_csv('datos_seguimiento_3_polyfit15.csv',sep=",")
array_datos = datos.to_numpy() #guarda los datos del csv cargado como arreglo de numpy

with open("mujoco29.xml", "r", encoding="utf-8") as f:
    xml = f.read() #Leer el archivo de mujoco cargado al entorno de ejecución y lo guarda en la variable xml


# Gym Enviroment

In [3]:
class RobotBipedoEnv(Env):
  def __init__(self, xml_text, tracking_array, tiempo_max_sim, tracking_fps=30):

    #Algunos parámetros asociados a la física del robot y la simulación de MuJoCo
    self.simulation_time_step=0.002 #La simulación corre en pasos de 0.002 segundos (frecuencia de 500 Hz)
    self.motor_velocity_deg_s=90 #La velocidad de los actuadores máxima es 90 deg/s **se modifica por reducciones
    self.motor_velocity_rad_s=(self.motor_velocity_deg_s)*(np.pi/180) #Conversión a rad/s
    self.max_step_motors=self.motor_velocity_rad_s*self.simulation_time_step #Traducción a máximo giro en radianes


    #Se crea la simulación con el xml cargado
    # Make model and data
    self.model = mujoco.MjModel.from_xml_string(xml_text)
    self.data = mujoco.MjData(self.model)


    # El estado es definido por 19 valores, donde se almacenan las coordenadas en 3 dimensiones de los 6 tracking points y el tiempo
    # Orden: Cadera pierna 1, Rodilla pierna 1, Tobillo pierna 1, Cadera pierna 2, Rodilla pierna 2, Tobillo pierna 2, tiempo
    # 0  Cadera pierna 1 x
    # 1  Cadera pierna 1 y
    # 2  Cadera pierna 1 z
    # 3  Rodilla pierna 1 x
    # 4  Rodilla pierna 1 y
    # 5  Rodilla pierna 1 z
    # 6  Tobillo pierna 1 x
    # 7  Tobillo pierna 1 y
    # 8  Tobillo pierna 1 z
    # 9  Cadera pierna 2 x
    # 10 Cadera pierna 2 y
    # 11 Cadera pierna 2 z
    # 12 Rodilla pierna 2 x
    # 13 Rodilla pierna 2 y
    # 14 Rodilla pierna 2 z
    # 15 Tobillo pierna 2 x
    # 16 Tobillo pierna 2 y
    # 17 Tobillo pierna 2 z
    # 18 tiempo
    self.observation_space = Box(low=-np.inf, high=np.inf, shape=(19, ))

    # Las acciones son definidas como un continuo en cada paso de la simulación. Limitado por el paso de la simulación y velocidad motores
    # Orden: Motor1_cadera_pierna1,Motor2_cadera_pierna1,Motor_rodilla_pierna1,Motor_tobillo_pierna1,Motor1_cadera_pierna2,Motor2_cadera_pierna2,Motor_rodilla_pierna2,Motor_tobillo_pierna2
    self.action_space = Box(low=-1, high=1, shape=(8,))

    #El estado inicial está dado por el tracking inicial
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time])))

    #Se configura un tiempo final de simulación
    if(tiempo_max_sim<=4.9):
      self.final_simulation_time=tiempo_max_sim
    else:
      self.final_simulation_time=4.9  #la simulación se termina en 5 segundos

    #Se almacena la secuencia de tracking de movimiento a seguir
    #Orden por columnas: Número de frame a X Hz, Ángulo cadera-rodilla_p1, Ángulo rodilla-tobillo_p1, Ángulo cadera-tobillo_p1, Ángulo cadera-rodilla_p2, Ángulo rodilla-tobillo_p2, Ángulo cadera-tobillo_p2 
    self.framerate=tracking_fps
    self.tracking=tracking_array

    #Crea una lista vacía para guardar frames de simulación (60 Hz) y otras opciones
    self.height=480
    self.width=640
    self.frames=[]
    self.renderer = mujoco.Renderer(self.model, self.height, self.width)

    #Inicializa el primer objetivo
    self.obj_frame=1


  #Función donde se accionan los controladores, según la acción y los límites
  def actuate_model(self, action):
    #Añade a los controladores los ángulos del vector de acciones
    self.data.actuator("motor1_cadera_pierna1").ctrl+=np.clip(action[0],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor2_cadera_pierna1").ctrl+=np.clip(action[1],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_rodilla_pierna1").ctrl+=np.clip(action[2],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_tobillo_pierna1").ctrl+=np.clip(action[3],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor1_cadera_pierna2").ctrl+=np.clip(action[4],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor2_cadera_pierna2").ctrl+=np.clip(action[5],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_rodilla_pierna2").ctrl+=np.clip(action[6],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_tobillo_pierna2").ctrl+=np.clip(action[7],-self.max_step_motors,self.max_step_motors)
    #Usa clip para aplicar los límites superiores e inferiores
    self.data.actuator("motor1_cadera_pierna1").ctrl=np.clip(self.data.actuator("motor1_cadera_pierna1").ctrl,-0.95,0.698)
    self.data.actuator("motor2_cadera_pierna1").ctrl=np.clip(self.data.actuator("motor2_cadera_pierna1").ctrl,-0.698,0.95)
    self.data.actuator("motor_rodilla_pierna1").ctrl=np.clip(self.data.actuator("motor_rodilla_pierna1").ctrl,0,1.48)
    self.data.actuator("motor_tobillo_pierna1").ctrl=np.clip(self.data.actuator("motor_tobillo_pierna1").ctrl,-0.31,0.61)
    self.data.actuator("motor1_cadera_pierna2").ctrl=np.clip(self.data.actuator("motor1_cadera_pierna2").ctrl,-0.95,0.698)
    self.data.actuator("motor2_cadera_pierna2").ctrl=np.clip(self.data.actuator("motor2_cadera_pierna2").ctrl,-0.698,0.95)
    self.data.actuator("motor_rodilla_pierna2").ctrl=np.clip(self.data.actuator("motor_rodilla_pierna2").ctrl,0,1.48)
    self.data.actuator("motor_tobillo_pierna2").ctrl=np.clip(self.data.actuator("motor_tobillo_pierna2").ctrl,-0.31,0.61)


  def step(self,action):

    #Ejecuta la acción y se cambian los controladores en la simulación
    self.actuate_model(action)
    #Da el paso de simulación
    mujoco.mj_step(self.model, self.data)
    #Lee las nuevas posiciones (estado)
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time])))

    #Calcula una reward, tiempo de simulación y objetivo de tracking de movimiento en frame específico
    #PENDIENTE: implementar cálculo de cuaterniones
    #PENDIENTE: implementar velocidad, end effector, center of mass
    #calcula los ángulos entre los puntos de tracking
    # 0  Cadera pierna 1 x
    # 1  Cadera pierna 1 y
    # 2  Cadera pierna 1 z
    # 3  Rodilla pierna 1 x
    # 4  Rodilla pierna 1 y
    # 5  Rodilla pierna 1 z
    # 6  Tobillo pierna 1 x
    # 7  Tobillo pierna 1 y
    # 8  Tobillo pierna 1 z
    # 9  Cadera pierna 2 x
    # 10 Cadera pierna 2 y
    # 11 Cadera pierna 2 z
    # 12 Rodilla pierna 2 x
    # 13 Rodilla pierna 2 y
    # 14 Rodilla pierna 2 z
    # 15 Tobillo pierna 2 x
    # 16 Tobillo pierna 2 y
    # 17 Tobillo pierna 2 z
    #*se multiplica por menos porque el eje coordenado y en MuJoCo está invertido con respecto al tracking
    current_angle_cadera_rodilla_pierna1=np.arctan((-(self.state[4]-self.state[1]))/np.abs((self.state[5]-self.state[2])))
    current_angle_rodilla_tobillo_pierna1=np.arctan((-(self.state[7]-self.state[4]))/np.abs((self.state[8]-self.state[5])))
    current_angle_cadera_tobillo_pierna1=np.arctan((-(self.state[7]-self.state[1]))/np.abs((self.state[8]-self.state[2])))
    current_angle_cadera_rodilla_pierna2=np.arctan((-(self.state[13]-self.state[10]))/np.abs((self.state[14]-self.state[11])))
    current_angle_rodilla_tobillo_pierna2=np.arctan((-(self.state[16]-self.state[13]))/np.abs((self.state[17]-self.state[14])))
    current_angle_cadera_tobillo_pierna2=np.arctan((-(self.state[16]-self.state[10]))/np.abs((self.state[17]-self.state[11])))
    #revisa el tiempo actual de simulación, contraste con los objetivos de la captura de movimiento, y revisa los ángulos objetivo
    if(self.data.time > self.obj_frame/self.framerate):
      self.obj_frame+=1
    objective_angle_cadera_rodilla_pierna1=self.tracking[self.obj_frame-1][1]
    objective_angle_rodilla_tobillo_pierna1=self.tracking[self.obj_frame-1][2]
    objective_angle_cadera_tobillo_pierna1=self.tracking[self.obj_frame-1][3]
    objective_angle_cadera_rodilla_pierna2=self.tracking[self.obj_frame-1][4]
    objective_angle_rodilla_tobillo_pierna2=self.tracking[self.obj_frame-1][5]
    objective_angle_cadera_tobillo_pierna2=self.tracking[self.obj_frame-1][6]
    #calcula reward
    rp=np.exp(-2*((objective_angle_cadera_rodilla_pierna1-current_angle_cadera_rodilla_pierna1)**2+(objective_angle_rodilla_tobillo_pierna1-current_angle_rodilla_tobillo_pierna1)**2+(objective_angle_cadera_rodilla_pierna2-current_angle_cadera_rodilla_pierna2)**2+(objective_angle_rodilla_tobillo_pierna2-current_angle_rodilla_tobillo_pierna2)**2))
    rv=0
    re=np.exp(-40*((objective_angle_cadera_tobillo_pierna1-current_angle_cadera_tobillo_pierna1)**2+(objective_angle_cadera_tobillo_pierna2-current_angle_cadera_tobillo_pierna2)**2))
    rc=0
    reward=0.65*rp+0.1*rv+0.15*re+0.1*rc

    #Condiciones de finalización
    #si pierde el equilibrio y se cae
    done=False
    if(self.state[2]<-0.2 or self.state[11]<-0.2):
      done=True
      reward=-25
    if(self.data.time >= self.final_simulation_time):
      done=True

    #Complementar
    info={
      "reward": reward,
      "state": self.state,
      "objective": self.tracking[self.obj_frame-1],
      "time": self.data.time
    }

    #Retornos
    return self.state, reward, done, info


  def render(self):
    #después de la función step(action) se llama a esta función, así que se comentan algunas líneas
    if len(self.frames) < self.data.time * self.framerate:
      self.renderer.update_scene(self.data, camera='camara_rodilla')
      pixels = self.renderer.render()
      self.frames.append(pixels)

  def reset(self):
    self.frames=[] #borra los frames
    self.obj_frame=1 #reinicia el objetivo de tracking
    mujoco.mj_resetData(self.model, self.data) #reinicia la simulación
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time]))) #cambia el estado tras el reinicio
    return self.state


# Creación del ambiente

In [4]:
env=RobotBipedoEnv(xml_text=xml, tracking_array=array_datos, tiempo_max_sim=2, tracking_fps=30)
#el ambiente es creado usando la variable xml donde se cargó la simulación y el arreglo con la captura de movimiento

# Entrenamiento con Stable Baselines 3

In [5]:
policy_kwargs = dict(activation_fn=th.nn.ReLU,net_arch=dict(pi=[1024, 512], vf=[1024, 512]))
model = PPO(MlpPolicy, env, policy_kwargs=policy_kwargs,  verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(

In [ ]:
# Crear el nombre del archivo CSV con timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')
csv_path = f"./logs/ppo_{timestamp}.csv"

# Crear un nuevo logger con CSV y salida en pantalla
new_logger = Logger(
    folder="./logs/",
    output_formats=[
        CSVOutputFormat(csv_path),  # Guardar en CSV con timestamp
        HumanOutputFormat(sys.stdout)  # Mostrar en el notebook
    ]
)

# Asignar el nuevo logger al modelo
model.set_logger(new_logger)

model.learn(total_timesteps=1_000_000, progress_bar=True)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | 264      |
| time/              |          |
|    fps             | 319      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 258          |
| time/                   |              |
|    fps                  | 277          |
|    iterations           | 2            |
|    time_elapsed         | 14           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0050947014 |
|    clip_fraction        | 0.0615       |
|    clip_range           | 0.2          |
|    entropy_loss         | -11.3        |
|    explained_variance   | -0.000725    |
|    learning_rate        | 0.0003       |
|    loss                 | 0.461        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00899     |
|    std                  | 0.993        |
|    value_loss           | 4            |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 258          |
| time/                   |              |
|    fps                  | 271          |
|    iterations           | 3            |
|    time_elapsed         | 22           |
|    total_timesteps      | 6144         |
| train/                  |              |
|    approx_kl            | 0.0075427485 |
|    clip_fraction        | 0.0721       |
|    clip_range           | 0.2          |
|    entropy_loss         | -11.3        |
|    explained_variance   | 0.701        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.467        |
|    n_updates            | 20           |
|    policy_gradient_loss | -0.00517     |
|    std                  | 0.995        |
|    value_loss           | 1.74         |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 257          |
| time/                   |              |
|    fps                  | 265          |
|    iterations           | 4            |
|    time_elapsed         | 30           |
|    total_timesteps      | 8192         |
| train/                  |              |
|    approx_kl            | 0.0052450374 |
|    clip_fraction        | 0.0312       |
|    clip_range           | 0.2          |
|    entropy_loss         | -11.3        |
|    explained_variance   | 0.867        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.12         |
|    n_updates            | 30           |
|    policy_gradient_loss | -0.00447     |
|    std                  | 0.995        |
|    value_loss           | 1.25         |
------------------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | 371         |
| time/                   |             |
|    fps                  | 249         |
|    iterations           | 485         |
|    time_elapsed         | 3974        |
|    total_timesteps      | 993280      |
| train/                  |             |
|    approx_kl            | 0.011136917 |
|    clip_fraction        | 0.156       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.15       |
|    explained_variance   | 0.999       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0339      |
|    n_updates            | 4840        |
|    policy_gradient_loss | -0.0101     |
|    std                  | 0.598       |
|    value_loss           | 0.0927      |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | 372         |
| time/                   |             |
|    fps                  | 249         |
|    iterations           | 486         |
|    time_elapsed         | 3982        |
|    total_timesteps      | 995328      |
| train/                  |             |
|    approx_kl            | 0.010723915 |
|    clip_fraction        | 0.136       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.13       |
|    explained_variance   | 0.999       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0253      |
|    n_updates            | 4850        |
|    policy_gradient_loss | -0.00713    |
|    std                  | 0.597       |
|    value_loss           | 0.112       |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | 373         |
| time/                   |             |
|    fps                  | 249         |
|    iterations           | 487         |
|    time_elapsed         | 3990        |
|    total_timesteps      | 997376      |
| train/                  |             |
|    approx_kl            | 0.011983525 |
|    clip_fraction        | 0.176       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.09       |
|    explained_variance   | 0.999       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0851      |
|    n_updates            | 4860        |
|    policy_gradient_loss | -0.00998    |
|    std                  | 0.594       |
|    value_loss           | 0.133       |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 373          |
| time/                   |              |
|    fps                  | 249          |
|    iterations           | 488          |
|    time_elapsed         | 3998         |
|    total_timesteps      | 999424       |
| train/                  |              |
|    approx_kl            | 0.0101563465 |
|    clip_fraction        | 0.153        |
|    clip_range           | 0.2          |
|    entropy_loss         | -7.05        |
|    explained_variance   | 0.999        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0325       |
|    n_updates            | 4870         |
|    policy_gradient_loss | -0.0111      |
|    std                  | 0.591        |
|    value_loss           | 0.0947       |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | 374         |
| time/                   |             |
|    fps                  | 249         |
|    iterations           | 489         |
|    time_elapsed         | 4006        |
|    total_timesteps      | 1001472     |
| train/                  |             |
|    approx_kl            | 0.011038858 |
|    clip_fraction        | 0.176       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.05       |
|    explained_variance   | 0.999       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00113     |
|    n_updates            | 4880        |
|    policy_gradient_loss | -0.0103     |
|    std                  | 0.592       |
|    value_loss           | 0.122       |
-----------------------------------------


In [7]:
model.save(f"ppo_{timestamp}")

In [8]:
eval_env=RobotBipedoEnv(xml_text=xml, tracking_array=array_datos, tiempo_max_sim=2)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:295.56 +/- 0.00


In [9]:
episodios=10
for i in range(0,episodios):
  state=eval_env.reset()
  new_state=state
  done=False
  recompensa_acumulada=0
  pasos=0

  while not done:
    state=new_state
    pasos+=1
    action, _states = model.predict(state, deterministic=True)
    new_state,reward,done,info=eval_env.step(action)
    recompensa_acumulada+=reward
  print("Episodio {}: Recompensa total {} en {} pasos de tiempo.".format(i+1,recompensa_acumulada,pasos))

Episodio 1: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 2: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 3: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 4: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 5: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 6: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 7: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 8: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 9: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.
Episodio 10: Recompensa total 295.5642008497905 en 1000 pasos de tiempo.


In [10]:
state=eval_env.reset()
new_state=state
done=False
recompensa_acumulada=0
pasos=0

while not done:
  state=new_state
  pasos+=1
  action, _states = model.predict(state, deterministic=True)
  print(action)
  new_state,reward,done,info=eval_env.step(action)
  recompensa_acumulada+=reward
  print(reward)
  eval_env.render()

print("Recompensa total {} en {} pasos de tiempo.".format(recompensa_acumulada,pasos))
media.show_video(eval_env.frames, fps=eval_env.framerate)

[-0.2305123   0.6133157   0.6536888   0.19803149 -0.0888886  -0.13932818
 -0.39112082 -0.60461676]
0.7336186404695142
[-0.0491883   1.          1.          0.00770389 -0.5231976  -0.58722794
 -0.7398188  -1.        ]
0.7336054606102309
[-0.05047955  1.          1.          0.00836065 -0.520245   -0.5853884
 -0.7387533  -1.        ]
0.7336192182367017
[-0.05177693  1.          1.          0.00901112 -0.51730025 -0.5835712
 -0.7377198  -1.        ]
0.7336982950871569
[-0.05309477  1.          1.          0.0096584  -0.5143434  -0.5817712
 -0.7367244  -1.        ]
0.7338569083819919
[-0.05443674  1.          1.          0.01030345 -0.5113696  -0.5799872
 -0.73576903 -1.        ]
0.734091690701604
[-0.0557973   1.          1.          0.01094485 -0.5083868  -0.5782218
 -0.73485255 -1.        ]
0.7343758457881399
[-0.05717278  1.          1.          0.01158229 -0.50539935 -0.5764748
 -0.7339716  -1.        ]
0.7346705079472652
[-0.05855912  1.          1.          0.01221573 -0.5024104  -0

This browser does not support the video tag.
